<u><h1>II- Demo. Optimal Interpolation </h1></u>

An example of simulated SSH data access is provided in the "example_data_access.ipynb" notebook. Here, an example of a mapping technique based on a simple optimal interpolation is proposed. The notebook is structured as follow: 

    1) reading of pseudo-observations,
    2) set optimal interpolation parameters and,
    3) perform optimal interpolation & save results.


Here, we assume a vector of observations, noted $y$ defined as:

$$y = H x + \epsilon $$

where $H$ is a linear observation operator between the reconstruction grid space and the observation space
, $x$ is the state to estimate and $\epsilon$ is an independent observation error.

The optimal interpolation consists in estimating an analysed state $x_{a}$ in combining the available observations to approximate the real state $x$:

$$x_{a} = K y $$
where $K$ is the weigth matrix defined as: 
\begin{equation*}
K = BH^T(HBH^T + R)^{−1}
\end{equation*}

$B$ is the covariance matrix of $x$, and $R$ the covariance matrix of the error vector $\epsilon$ ($^T$ is the transpose operator)

In [ ]:
import xarray as xr
import numpy
import pyinterp
import dask
import warnings
import logging
import sys
import os
warnings.filterwarnings('ignore')

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:63066 Dashboard: http://127.0.0.1:63065/status,Cluster Workers: 4 Cores: 8 Memory: 17.04 GB


In [8]:
sys.path.append('..')

In [9]:
from src.mod_oi import *
from src.mod_inout import *
from src.mod_regrid import *
from src.mod_eval import *
from src.mod_plot import *

### OI parameters

In [10]:
# OI Grid
lon_min = 295.                                           # domain min longitude
lon_max = 305.                                           # domain max longitude
lat_min = 33.                                            # domain min latitude
lat_max = 43.                                            # domain max latitude
time_min = numpy.datetime64('2012-10-22')                # domain min time
time_max = numpy.datetime64('2012-12-02')                # domain max time
dx = 0.2                                                 # zonal grid spatial step (in degree)
dy = 0.2                                                 # meridional grid spatial step (in degree)
dt = numpy.timedelta64(1, 'D')                           # temporal grid step

simu_start_date = '2012-10-01T00:00:00'                  # Nature run initial date

glon = numpy.arange(lon_min, lon_max + dx, dx)           # output OI longitude grid
glat = numpy.arange(lat_min, lat_max + dy, dy)           # output OI latitude grid
gtime = numpy.arange(time_min, time_max + dt, dt)        # output OI time grid

# OI parameters
Lx = 1.                                                  # Zonal decorrelation scale (in degree)
Ly = 1.                                                  # Meridional decorrelation scale (in degree)
Lt = 7.                                                  # Temporal decorrelation scale (in days)
noise = 0.05                                             # Noise level (5%)

### Define IO folders

In [14]:
inputs = ['../dc_obs/BOOST-SWOT_jason1_2012-10-01_2013-09-29.nc']

In [35]:
output_directory = '../results/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)  
output_oi = f'../results/ssh_reconstruction_{time_min}-{time_max}_jason1.nc'
output_oi_regridded = f'../results/ssh_reconstruction_regridded_{time_min}-{time_max}_jason1.nc'

In [36]:
ds = xr.open_dataset(inputs[0])
ds

<xarray.Dataset>
Dimensions:      (cycle: 1, time: 52717)
Coordinates:
  * time         (time) datetime64[ns] 2012-10-01T03:20:37.213526 ... 2013-09-29T05:35:56.659327
Dimensions without coordinates: cycle
Data variables:
    lon          (time) float64 ...
    lat          (time) float64 ...
    x_al         (time) float32 ...
    ncycle       (time, cycle) timedelta64[ns] ...
    model_index  (time) float32 ...
    ssh_obs      (time) float64 ...
    ssh_model    (time) float64 ...
    nadir_err    (time) float64 ...
Attributes:
    description:               SWOT fixed grid
    corresponding_grid:        
    title:                     Altimeter like data simulated by SWOT simulator
    keywords:                  check keywords
    Conventions:               CF-1.6
    summary:                   SWOT grid data produced
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    history:                   Grid File created by swotsimulator version 2.21
    processing_level:          L2
    standard_name_vocabulary:  CF-1.6
    creator_name:              Lucile Gaultier and Clement Ubelmann
    creator_email:             lucile.gaultier@gmail.com
    publisher_url:             github/SWOTSimulator/
    time_coverage_start:       0.3383046
    time_coverage_end:         9.895123
    geospatial_lat_min:        26.46
    geospatial_lat_max:        66.15
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        360.00
    geospatial_lon_min:        0.00
    geospatial_lon_units:      degrees_east
    project:                   SWOT
    date_created:              2018-10-12T12:39:50Z
    date_modified:             2018-10-12T12:39:50Z
    keywords_vocabulary:       NASA
    references:                Gaultier, L., C. Ubelmann, and L.-L. Fu, 2016:...

In [37]:
# Note: dc_ref is used for reggriding step
dc_ref = xr.open_mfdataset('../dc_ref/*.nc', combine='nested', concat_dim='time', parallel=True)
dc_ref

,Array,Chunk
Bytes,12.61 GB,34.56 MB
Shape,"(8760, 600, 600)","(24, 600, 600)"
Count,1095 Tasks,365 Chunks
Type,float32,numpy.ndarray


In [38]:
# Note: dc_ref_sample is used for reggriding step (daily mean is enougth !!!!!)
dc_ref_sample = dc_ref.sel(time=slice(time_min, time_max)).resample(time='1D').mean()
del dc_ref
dc_ref_sample

,Array,Chunk
Bytes,59.04 MB,1.44 MB
Shape,"(41, 600, 600)","(1, 600, 600)"
Count,1341 Tasks,41 Chunks
Type,float32,numpy.ndarray


### OI core

In [39]:
%%time
# set OI param & grid
ds_oi1_param = oi_param(Lx, Ly, Lt, noise)
ds_oi1_grid = oi_grid(glon, glat, gtime, simu_start_date)
# Read input obs + discard a bit...
coarsening = {'time': 5}
ds_oi1_obs = read_obs(inputs, ds_oi1_grid, ds_oi1_param, simu_start_date, coarsening)
# Run OI
for it in range(len(gtime)):
    oi_core(it, ds_oi1_grid, ds_oi1_param, ds_oi1_obs)
# Regrid    
ds_oi1_regrid = oi_regrid(ds_oi1_grid, dc_ref_sample)

INFO:root:     Set OI params...
INFO:root:     Set OI grid...
INFO:root:     Reading observations...


INFO:root:     Regridding...


Wall time: 21.7 s


### Save output (raw OI + regridded fields)

In [40]:
# Save raw OI reconstruction
ds_oi1_grid.to_netcdf(output_oi)

In [41]:
# Save OI reconstruction regridded on reference grid
ds_oi1_regrid.to_netcdf(output_oi_regridded)